In [1]:
import requests
from bs4 import BeautifulSoup
from monkeylearn import MonkeyLearn
from googlesearch import search
import pandas as pd
import time

df = pd.DataFrame(columns = ['Company','Ticker','Year','Quarter',
                             'Date','URL',
                             'Sentiment1','Confidence1',
                             'Sentiment2','Confidence2'])

companies = ['facebook','alphabet','tesla','citigroup']
years = ['2019']
quarters = ['q4','q3','q2','q1']
fixed_key_word = 'earnings call trascript motley fool'

In [4]:
!pip install google

In [2]:
def get_url(query,j,k):
    j1 = 0
    indicator = ''
    for i1 in search(query, tld = 'com', num = 5, stop = 5, pause = 2.0):
        if j in i1 and k in i1:
            indicator = i1
            break
    return indicator

def get_content(url):
    page = requests.get(url)
    soup1 = BeautifulSoup(page.content, 'html.parser')   
    return soup1
   
def get_ticker(soup1):
    tick = soup1.find_all('span', class_ = "ticker")
    ticker = tick[0].get_text().split(':')[1]
    return ticker

# Here we extract the infromation about the name of this report, including the time
def get_name_date_time(soup1):
    # br tag here
    title = soup1.find_all('br')
    time_ = ' '
    
    if title[1].find_all('span') == []: 

        count = title[1].get_text().split('Call')[1].count(',')
        if count == 2: 
            date_time = title[1].get_text().split('Call')[1]
            date = ','.join([date_time.split(',')[0],date_time.split(',')[1]]) 
            time_ = date_time.split(',')[2]
        else: 
            date_time = title[1].get_text().split('Call')[1]
            date = ' '.join([date_time.split(' ')[0],date_time.split(' ')[1],date_time.split(' ')[2]])
            time_ = ' '.join([date_time.split(' ')[3],date_time.split(' ')[4],date_time.split(' ')[5]])
    else:
        date = title[1].find_all('span')[0].get_text()
        time_ = title[1].find_all('em')[0].get_text()

    name = title[1].get_text().split(date)[0]
  
    info = [name, date, time_]
    return info


def get_text(soup1):
    company = soup1.find_all('span', class_ = "article-content")
    # We can get the text out of the entire article and the data type will be str
    text = company[0].get_text()
    
    # Vaiours scenarios here
    key_words1 = 'Questions and Answers:'
    key_words2 = '[Operator Closing Remarks]'
    if key_words1 not in text:
        key_words1 = 'Questions & Answers:'
        if key_words1 not in text:
            key_words1 = 'Questions & Answerss:'
            if key_words1 not in text:
                key_words1 = 'Questions and Answerss:'
    if key_words2 not in text:
        key_words2 = 'Duration:'

    text_speakings = text.split('Prepared Remarks:')[1].split(key_words1)[0]
    text_QA = text.split('Prepared Remarks:')[1].split(key_words1)[1].split(key_words2)[0] 
    text1 = [text_speakings,text_QA] 
    return text1

def AP1_sentiment_analysis(text):
    
    # My API of MonkeyLearn, which is subject to change later
    ml = MonkeyLearn('b9c4aa1490f2bbdaba1fc0d72c67437336e77799')

    # Model_id will stay the same 
    model_id = 'cl_pi3C7JiL'
    result = ml.classifiers.classify(model_id, text)
    outcome = result.body
    Pos_Neg = outcome[0].get('classifications')[0].get('tag_name')
    Confidence = outcome[0].get('classifications')[0].get('confidence')
    result1 = (Pos_Neg,Confidence)
    return result1

def get_outcome(text1):
    
    x1 = [" ".join(text1[0])]
    x2 = [" ".join(text1[1])]
    
    result1 = AP1_sentiment_analysis(x1)
    result2 = AP1_sentiment_analysis(x2)
    
    outcome = {'part1':result1,'part2':result2}
    
    return outcome

In [3]:
for i in companies: 
    for j in years:
        for k in quarters:
            query = ' '.join([i,j,k,fixed_key_word])
            url = get_url(query,j,k)
            print(i,j,k,url)
            
            soup1 = get_content(url)
            text1 = get_text(soup1)
            text = text1[0]+text1[1]
            m = j+' '+k+' '+i
            with open('/Users/zhangluyang/Desktop/ECT/{} ECT.txt'.format(m),'w') as file:
                file.write(text)
                file.close()
            
            
            info = get_name_date_time(soup1)
            info1 = info[0].split()
            print(info1[1],' ',info1[0],'',info[1],' ',info[2])
            
            ticker = get_ticker(soup1)
            ticker = ticker.replace('(','').replace(')','')
            
            outcome = get_outcome(text1)
            outcome1 = outcome.get('part1')
            outcome2 = outcome.get('part2')
            
            list_to_add = [i,ticker, info1[1], info1[0],
                           info[1], url,
                           outcome1[0], outcome1[1],
                           outcome2[0], outcome2[1]]

            a_series = pd.Series(list_to_add, index = df.columns)
            df = df.append(a_series, ignore_index = True)
            time.sleep(0.5)
            
df.to_csv(r'/Users/zhangluyang/Desktop/ECT/table.csv')

facebook 2019 q4 https://www.fool.com/earnings/call-transcripts/2020/01/29/facebook-inc-fb-q4-2019-earnings-call-transcript.aspx
2019   Q4  Jan 29, 2020   5:00 p.m. ET
facebook 2019 q3 https://www.fool.com/earnings/call-transcripts/2019/10/31/facebook-inc-fb-q3-2019-earnings-call-transcript.aspx
2019   Q3  Oct 30, 2019   5:00 p.m. ET
facebook 2019 q2 https://www.fool.com/earnings/call-transcripts/2019/07/24/facebook-inc-fb-q2-2019-earnings-call-transcript.aspx
2019   Q2  Jul 24, 2019   5:00 p.m. ET
facebook 2019 q1 https://www.fool.com/earnings/call-transcripts/2019/04/25/facebook-inc-fb-q1-2019-earnings-call-transcript.aspx
2019   Q1  April 24, 2019   5:00 p.m. ET
alphabet 2019 q4 https://www.fool.com/earnings/call-transcripts/2020/02/03/alphabet-inc-goog-googl-q4-2019-earnings-call-tran.aspx
2019   Q4  Feb. 03, 2020   5:00 p.m. ET


KeyboardInterrupt: 

# Only serves as testing

url = 'https://www.fool.com/earnings/call-transcripts/2018/04/26/microsoft-corporation-msft-q3-2018-earnings-confer.aspx'
soup1 = get_content(url)

company = soup1.find_all('span', class_ = "article-content")
text = company[0].get_text()

key_words1 = 'Questions and Answers:'
key_words2 = '[Operator Closing Remarks]'
if key_words1 not in text:
    key_words1 = 'Questions & Answers:'
    if key_words1 not in text:
        key_words1 = 'Questions and Answerss:'
if key_words2 not in text:
    key_words2 = 'Duration:'

text_speakings = text.split('Prepared Remarks:')[1].split(key_words1)[0]
#print(text_speakings)
#print(text.split('Prepared Remarks:')[1].split(key_words1)[1])
text_QA = text.split('Prepared Remarks:')[1].split(key_words1)[1].split(key_words2)[0]
    #print(len(text_QA))
    #print(type(text_QA))
    #print(text_QA)   
text1 = [text_speakings,text_QA] 



title = soup1.find_all('br')
print(title[1])    
if title[1].find_all('span') == []: 

    count = title[1].get_text().split('Call')[1].count(',')
    if count == 2: 
        date_time = title[1].get_text().split('Call')[1]
        print(date_time)
        date = ','.join([date_time.split(',')[0],date_time.split(',')[1]]) 
        time_ = date_time.split(',')[2]
        print(time_)
    else: 
        date_time = title[1].get_text().split('Call')[1]
        date = ' '.join([date_time.split(' ')[0],date_time.split(' ')[1],date_time.split(' ')[2]])
        time = ' '.join([date_time.split(' ')[3],date_time.split(' ')[4],date_time.split(' ')[5]])
else:
    date = title[1].find_all('span')[0].get_text()
    time_ = title[1].find_all('em')[0].get_text()
    #date = title[1].find_all('span')[0].get_text()
print(date,'---',time_)
#info = get_name_date_time(soup1)
#info1 = info[0].split()